# Interactive Workflow Development with Jupyter

This notebook demonstrates the correct way to run `matterlab-opentrons` workflows within a Jupyter Notebook or JupyterLab environment.

## Important: Asyncio Compatibility

Because both Jupyter and Prefect run their own asynchronous event loops, you need to apply a patch to allow them to work together. The `nest_asyncio` library provides this patch.

## Step 1: Apply the Asyncio Patch

Run the following cell once at the beginning of your notebook session. This allows Prefect's async event loop to run inside Jupyter's.

In [ ]:
import nest_asyncio
nest_asyncio.apply()
print("✅ Asyncio patch applied successfully!")

## Step 2: Import Your Dependencies

Now you can import all the necessary components from Prefect and your Opentrons library.

In [ ]:
from prefect import flow
from matterlab_opentrons import connect
from matterlab_opentrons.prefect_tasks import robust_task
print("✅ Dependencies imported successfully!")

## Step 3: Define Your Tasks

As you would in a normal script, use the `@robust_task` decorator to create tasks for each robot action you intend to use. This is where you can interactively build up the components of your protocol.

In [ ]:
@robust_task(retries=2)
def pick_up_tip(pipette, *args, **kwargs):
    """Pick up a tip with retry logic."""
    return pipette.pick_up_tip(*args, **kwargs)

@robust_task
def aspirate(pipette, *args, **kwargs):
    """Aspirate liquid from a well."""
    return pipette.aspirate(*args, **kwargs)

@robust_task
def dispense(pipette, *args, **kwargs):
    """Dispense liquid to a well."""
    return pipette.dispense(*args, **kwargs)

@robust_task
def drop_tip(pipette, *args, **kwargs):
    """Drop the current tip."""
    return pipette.drop_tip(*args, **kwargs)

print("✅ Tasks defined successfully!")

## Step 4: Define and Run Your Flow

Now you can define a `@flow` and call it directly in the same cell. Prefect will execute the protocol, and you will see the logs output directly below the cell. This allows for rapid testing and iteration of your protocol logic.

In [ ]:
@flow
def my_interactive_protocol(simulation: bool = True):
    """An example flow to run interactively."""
    robot = None
    try:
        # Use the new connect() function with simulation alias
        robot = connect(host_alias="ot2_sim", simulation=simulation)
        
        # Load labware and instruments using the new API
        plate = robot.deck.load_labware("corning_96_wellplate_360ul_flat", location="1")
        tip_rack = robot.deck.load_labware("opentrons_96_tiprack_300ul", location="2")
        pipette = robot.load_pipette("p300_single_gen2", mount="right")
        
        # Run a simple transfer
        print("Running a simple transfer...")
        pick_up_tip(pipette)
        aspirate(pipette, volume=50, location=plate["A1"])
        dispense(pipette, volume=50, location=plate["B1"])
        drop_tip(pipette)
        print("Transfer complete.")

    finally:
        if robot:
            print("Closing robot connection.")
            robot.close()

# --- Run the flow ---
print("🚀 Starting interactive protocol...")
my_interactive_protocol(simulation=True)

## Step 5: Customize Your Protocol

You can now modify the protocol above or create new cells to test different operations. Here are some examples:

In [ ]:
# Example: Serial dilution protocol
@flow
def serial_dilution_demo(simulation: bool = True):
    """Demonstrate a simple serial dilution."""
    robot = None
    try:
        robot = connect(host_alias="ot2_sim", simulation=simulation)
        
        # Load labware
        source_plate = robot.deck.load_labware("corning_96_wellplate_360ul_flat", location="1")
        dilution_plate = robot.deck.load_labware("corning_96_wellplate_360ul_flat", location="2")
        tip_rack = robot.deck.load_labware("opentrons_96_tiprack_300ul", location="3")
        pipette = robot.load_pipette("p300_single_gen2", mount="right")
        
        # Perform serial dilution
        print("Performing 1:2 serial dilution...")
        for i in range(8):  # Columns A-H
            col = chr(65 + i)  # A, B, C, ..., H
            
            # Transfer from source to dilution plate
            pick_up_tip(pipette)
            aspirate(pipette, volume=100, location=source_plate[f"{col}1"])
            dispense(pipette, volume=100, location=dilution_plate[f"{col}1"])
            drop_tip(pipette)
            
            # Add diluent
            pick_up_tip(pipette)
            aspirate(pipette, volume=100, location=source_plate[f"{col}2"])
            dispense(pipette, volume=100, location=dilution_plate[f"{col}1"])
            drop_tip(pipette)
        
        print("Serial dilution complete!")
        
    finally:
        if robot:
            robot.close()

# Uncomment to run:
# serial_dilution_demo(simulation=True)

## Step 6: Flex Robot Example

Here's an example using the Flex robot with gripper functionality:

In [ ]:
@flow
def flex_gripper_demo(simulation: bool = True):
    """Demonstrate Flex robot with gripper functionality."""
    robot = None
    try:
        # Connect to Flex robot
        robot = connect(host_alias="flex_sim", simulation=simulation)
        
        # Load labware
        plate = robot.deck.load_labware("corning_96_wellplate_360ul_flat", location="A1")
        tip_rack = robot.deck.load_labware("opentrons_96_tiprack_300ul", location="A2")
        
        # Load instruments
        pipette = robot.load_pipette("p300_single_flex", mount="left")
        gripper = robot.load_gripper()
        
        # Demonstrate gripper operations
        print("Moving gripper to plate location...")
        gripper.move_to(plate.location)
        
        print("Gripping plate...")
        gripper.grip()
        
        print("Moving to new location...")
        gripper.move_to(robot.deck.get_slot("A3"))
        
        print("Releasing plate...")
        gripper.ungrip()
        
        # Demonstrate pipette operations
        print("Performing liquid transfer...")
        pick_up_tip(pipette)
        aspirate(pipette, volume=50, location=plate["A1"])
        dispense(pipette, volume=50, location=plate["B1"])
        drop_tip(pipette)
        
        print("Flex demo complete!")
        
    finally:
        if robot:
            robot.close()

# Uncomment to run:
# flex_gripper_demo(simulation=True)

## Tips for Interactive Development

1. **Always use `simulation=True`** during development to avoid accidents
2. **Test small operations first** before building complex protocols
3. **Use the `@robust_task` decorator** for automatic retry logic
4. **Check your `user_scripts/sshclient_settings.json`** file to ensure your robot aliases are correct
5. **Restart the kernel** if you encounter asyncio-related errors
6. **Logs are automatically saved** to `user_scripts/logs/` with timestamps
7. **Use the new `connect()` function** instead of direct class instantiation

## Configuration Files

All configuration files are now located in the `user_scripts/` directory:
- `sshclient_settings.json` - Robot connection settings
- `labware_definitions.py` - Custom labware definitions
- `labware_template.json` - Labware generation template
- `prefect_setup.py` - Prefect environment setup helper

## Next Steps

Once you're satisfied with your protocol in the notebook, you can:

1. Copy the flow definition to a Python script
2. Set `simulation=False` to run on a real robot
3. Add more sophisticated error handling and logging
4. Integrate with other Prefect features like scheduling and monitoring
5. Generate custom labware using the `labware_generator.py` script